In [1]:
import pandas as pd
import requests
import time
import tqdm
import io

In [5]:
# appelle du département par code postal
api = 'eyJ4NXQiOiJZV0kxTTJZNE1qWTNOemsyTkRZeU5XTTRPV014TXpjek1UVmhNbU14T1RSa09ETXlOVEE0Tnc9PSIsImtpZCI6ImdhdGV3YXlfY2VydGlmaWNhdGVfYWxpYXMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJnZGxlZHMzMUBjYXJib24uc3VwZXIiLCJhcHBsaWNhdGlvbiI6eyJvd25lciI6ImdkbGVkczMxIiwidGllclF1b3RhVHlwZSI6bnVsbCwidGllciI6IlVubGltaXRlZCIsIm5hbWUiOiJEZWZhdWx0QXBwbGljYXRpb24iLCJpZCI6MjgxOTcsInV1aWQiOiJjMDA0ZmQ1NC0xMTY3LTQ3MTEtOWQ3MC04M2ExZWI0YmI0MGYifSwiaXNzIjoiaHR0cHM6XC9cL3BvcnRhaWwtYXBpLm1ldGVvZnJhbmNlLmZyOjQ0M1wvb2F1dGgyXC90b2tlbiIsInRpZXJJbmZvIjp7IjUwUGVyTWluIjp7InRpZXJRdW90YVR5cGUiOiJyZXF1ZXN0Q291bnQiLCJncmFwaFFMTWF4Q29tcGxleGl0eSI6MCwiZ3JhcGhRTE1heERlcHRoIjowLCJzdG9wT25RdW90YVJlYWNoIjp0cnVlLCJzcGlrZUFycmVzdExpbWl0IjowLCJzcGlrZUFycmVzdFVuaXQiOiJzZWMifX0sImtleXR5cGUiOiJQUk9EVUNUSU9OIiwic3Vic2NyaWJlZEFQSXMiOlt7InN1YnNjcmliZXJUZW5hbnREb21haW4iOiJjYXJib24uc3VwZXIiLCJuYW1lIjoiRG9ubmVlc1B1YmxpcXVlc0NsaW1hdG9sb2dpZSIsImNvbnRleHQiOiJcL3B1YmxpY1wvRFBDbGltXC92MSIsInB1Ymxpc2hlciI6ImFkbWluX21mIiwidmVyc2lvbiI6InYxIiwic3Vic2NyaXB0aW9uVGllciI6IjUwUGVyTWluIn1dLCJleHAiOjE3NDk0NjAxNzcsInRva2VuX3R5cGUiOiJhcGlLZXkiLCJpYXQiOjE3NDk0NTY1NzcsImp0aSI6IjNlMDE2MzBlLTc2MmQtNDZhMi1iYzNmLWIwOTEyYWE2OTkxNSJ9.VePR6hNpqzz9dagiTlHkaXQ8Wb9A7rLbvgfTgtey-ytjFjTnUYfkLExUIYtT_WO3ksImYSRMpg2Oc2OEQLQxR97632nsG4ttOL-oDknfd7ahGf4yzFOPJI_Sqgw6NyiCYX23p4r3RR18BH1OsG6p07rOhtm9LEbpMyIGOucaROVpIqZuJAFLKr7Z94gPGOY8-buvtHiV9o9nUSok5veEcK4gDTZnafEhXyI3_TRtzeoZSXI4ZZY_nrWCOlfwilaB4S8uT7E2k9xezsxjTwp_ird57DMoLbLz_kxyalotD2X7BKrkW3i33mE2iQkAC4cdgMmAEvwk6njF0CmSo0hd-w=='
dep = '83'
url = 'https://public-api.meteofrance.fr/public/DPClim/v1/liste-stations/quotidienne'

params = {
    'id-departement': dep,
    'parametre': 'temperature'
}

headers = {
    'accept': '*/*',
    'apikey': api
}

corse = requests.get(url, headers=headers, params=params)

print('erreur corse', corse.status_code)
corse.json()

# on applique un mask pour ne prendre que les stations ouvertes
corse_df = pd.DataFrame(corse.json())
corse_df['posteOuvert'] = corse_df['posteOuvert'].astype(bool)
mask = corse_df['posteOuvert'] == True
corse_df = corse_df[mask]

# appelle les information par station sur plusieurs années
# all_data = pd.DataFrame()

id = corse_df['id']

for i in id:

    for années in range(2025, 2026):
        date_debut = f'{années}-01-01T00:00:00Z'
        date_fin = f'{années}-06-05T23:59:59Z'

        url = "https://public-api.meteofrance.fr/public/DPClim/v1/commande-station/quotidienne"
        params = {
            "id-station": i,
            "date-deb-periode": date_debut,
            "date-fin-periode": date_fin
        }
        headers = {
            "accept": "*/*",
            "apikey": api
        }

        corse1 = requests.get(url, headers=headers, params=params)
        print('erreur corse1', corse1.status_code)
        corse1_json = corse1.json()

        # Wrap in list if it's a dict of scalars
        corse1_df = pd.DataFrame(corse1_json).reset_index()

        name = dep + '_' + corse_df.loc[corse_df['id'] == i, 'nom'].values[0] + '_' + str(années)
       
        # Extract the 'return' value if the response is a dict
        id_cmde = corse1_df.iloc[0,1]
       

        url = "https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier"
        params = {
            "id-cmde": id_cmde
        }
        headers = {
            "accept": "*/*",
            "apikey": api
        }
        corse2 = requests.get(url, headers=headers, params=params)
        
        print('erreur corse2', corse2.status_code)
        #print(corse2.text)
        
        # output_file = name +'.csv'
        # pd.DataFrame(corse2).to_csv(output_file, index=False)

        nom_station = corse_df.loc[corse_df['id'] == i, 'nom'].values[0].replace(' ', '_')
        output_file = f"{dep}/{dep}_{nom_station}_{années}.csv"

        # Enregistrement du fichier
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(corse2.text)

        time.sleep(60 / 25)  # 60 seconds divided by 25 requests
# output_file = dep + '.csv'
# all_data.to_csv(output_file, index=False)

erreur corse 200
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur corse1 202
erreur corse2 201
erreur cors